## Q-Learning with Neural Networks in OpenAI Gym Environment

Import Libraries

In [1]:
import gym
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline

Load OpenAI Gym "FrozenLake" Environment 

In [2]:
env = gym.make("FrozenLake-v0")

[2018-01-26 22:41:11,475] Making new env: FrozenLake-v0


Implementing Q-Learning algorithm with Neural Network approach

In [3]:
tf.reset_default_graph()

In [4]:
#Let's construct a feed forward network which takes one-hot encoded input states as input and outputs the actions to be taken
inputs = tf.placeholder(shape=[1,16], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4], minval=0, maxval=0.01))
Qout = tf.matmul(inputs, W)
predict = tf.argmax(Qout, 1)

#The objective function is minimized by computing the loss which is the sum of square difference between the target and prediction Q values
nextQ = tf.placeholder(shape=[1,4], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [5]:
np.identity(16)[1:2]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.]])

Training the network

In [ ]:
init = tf.global_variables_initializer()

#learning parameters
y = 0.99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #On every iteration through the episode we reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j += 1
            #Choose an action greedily(with e chance of random function) from the Q-Network
            a, allQ = sess.run([predict, Qout], feed_dict={inputs: np.identity(16)[s:s+1]})
            if(np.random.rand(1) < e):
                a[0] = env.action_space.sample()
            #Get new state and reward from the environment
            s1, r, d, _ = env.step(a[0])
            #We get the Q-values by feeding the new state through our network
            Q1 = sess.run(Qout, feed_dict={inputs: np.identity(16)[s1:s1+1]})
            #Obtain maxQ and set our target value for choosen action
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _, W1 = sess.run([updateModel, W], feed_dict={inputs: np.identity(16)[s:s+1], nextQ: targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)

print("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")

We observe that the FrozenLake problem when solved through neural networks isnt efficient when compared to Q-Table approach.(This can improved by using methods such as Experience Replay and Freezing Target Networks.